In [1]:
import csv
import json
import requests
import pprint

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from dateutil.parser import parse
from sklearn import preprocessing
from sklearn.cluster import KMeans

%matplotlib inline



In [2]:

pd.set_option('display.max_columns', None)
data = pd.read_csv('Second_half_2019_over0.csv', header=[0,1], index_col=0)

/Users/kimyoungwoo/Library/Python/3.7/lib/python/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
data.columns

MultiIndex([(                 'l1',                             'l2'),
            (                 'AP',                     'IP Address'),
            (                 'AP',                    'MAC Address'),
            (                 'AP',                   'Map Location'),
            (                 'AP',                           'Name'),
            (             'Client',                  'Endpoint Type'),
            (             'Client',                     'IP Address'),
            (             'Client',                    'MAC Address'),
            (             'Client',                           'SSID'),
            (   'Session Duration',                           'days'),
            (   'Session Duration',                          'hours'),
            (   'Session Duration',                        'minutes'),
            (   'Session Duration',                        'seconds'),
            (        'Utilization', 'Avg. Session Throughput (Kbps)'),
      

In [43]:
def preprocessing(data):
    data[('Disassociation Time','Unnamed: 23_level_1')] = pd.to_datetime(data[('Disassociation Time','Unnamed: 23_level_1')])
    data[('Association Time','Unnamed: 24_level_1')] = pd.to_datetime(data[('Association Time','Unnamed: 24_level_1')])
    buildings = list(set(data[('AP','Map Location')]))
    building_floor = {}
    for a in buildings:
        b = a.split(') >')
        if len(b) == 1 :
            continue
        b[0] += ')'
        if b[0] not in building_floor :
            building_floor[b[0]] = list()
            building_floor[b[0]].append(b[1])
        else :
            building_floor[b[0]].append(b[1])
    return building_floor, data

In [44]:

def find_buildingdata(building, data, building_floor):
    chosen = []
    for floor in building_floor[building]:
        ss = building + ' >' + floor
        chosen.append(data[data[('AP','Map Location')] == ss]) 
    return pd.concat(chosen)

In [54]:
def find_11(building_data):
    express = ['Mon', 'Tue', 'Wed', 'Thr', 'Fri', 'Sat', 'Sun']
    for i in range(7):
        info = building_data[building_data['Association Time']['Unnamed: 24_level_1'].dt.weekday == i]
        APS = info[('AP', 'Name')].value_counts()
        if APS.index[0].startswith('AP') or APS.index[0].startswith('11') :
            print(APS.index[0], express[i])

# 먼저 datetime으로 바꿔야 안터져서 바꾸고 인덱스 만들고 이래야 빠름

In [55]:
after, data = preprocessing(data)


In [56]:
for building in after :
    print(building)
    building_data = find_buildingdata(building, data, after)
    if building_data.shape[0] == 0:
        continue
    #print(type(building_data.iloc[0][('Association Time','Unnamed: 24_level_1')]))
    find_11(building_data)

Pusan > JaDae (313)
Pusan > 3Gong (105)
11S-105-2-4 Sun
Pusan > SungHak (422)
11S-422-7-3 Mon
11S-422-4-3 Tue
11S-422-7-3 Wed
11S-422-4-3 Thr
11S-422-7-3 Fri
11S-422-7-3 Sat
11S-422-7-3 Sun
Pusan > TohabGiGe (303)
Pusan > MulRy1 (308)
11S-308-1-3 Fri
Pusan > JaeRyo (405)
11S-405-3-3 Fri
11S-405-3-3 Sat
11S-405-3-3 Sun
Pusan > JaYoun (411)
Pusan > HwaHak (606)
11S-606-2-2 Mon
11S-606-4-4 Tue
11S-606-4-4 Wed
11S-606-4-4 Thr
11S-606-2-2 Fri
11S-606-4-4 Sat
11S-606-4-4 Sun
Pusan > JoSoSil (702)
Pusan > GunSul (401)
AP-175.217 Thr
Pusan > KongYounDong (607)
Pusan > T_Gong (207)
Pusan > Kyosu1 (307)
Pusan > MulRy2 (309)
Pusan > GiGoo (414)
Pusan > 2SaBeom (701)
Pusan > KyungYoung (514)
11S-514-1-6 Mon
AP-514-4-6 Tue
11S-514-1-6 Wed
AP-514-4-6 Thr
11S-514-1-6 Fri
11S-514-1-6 Sat
11S-514-1-6 Sun
Pusan > Inmoon (306)
Pusan > BonKwan (205)
Pusan > JeDo (201)
Pusan > NewJunsan (515)
Pusan > Silhum (111)
Pusan > SaengMul (416)
AP-88.217 Mon
AP-88.217 Tue
AP-88.217 Wed
AP-88.217 Thr
AP-88.217 Sat
P